# 爬蟲：工商時報

- 讀取檔案 (google_search_result_list_TMC.csv)
- 抓取文章內容
- 取出日期,標題,內文

In [ ]:
import requests
import time
from bs4 import BeautifulSoup

In [ ]:
def get_news_date(soup):
    news_date = '' 
    node = soup.find(class_="post-published updated")
    if (node == None):
        node = soup.find(class_="post-meta-date")
    if (node != None):
        news_date = node.text.strip()
        if ' ' in news_date:
            news_date = news_date.split(' ')[0]
    else:
        print('no date')
    return news_date

In [ ]:
def get_news_title(soup):
    node = soup.find('title')
    if node != None:
        return node.text.strip()
    else:
        print('no title')
        return('')

In [ ]:
def get_news_body(soup):
    body = '';
    node = soup.find(class_='entry-content clearfix single-post-content')
    if node == None:
        node = soup.find(class_='entry-content')
    if node != None:
        p_list = node.find_all('p')
        for p in p_list:
            body = body + p.text.strip()
    else:
        print('no content')
    return body

In [ ]:
def fetch_page(url):
    r = None
    try:
        r = requests.get(url,timeout=3)
    except requests.exceptions.RequestException as e:
        print(e)
    if r.status_code == requests.codes.ok:
        #r.encoding = "UTF-8"
        soup = BeautifulSoup(r.text, 'html.parser')
        #print(soup.prettify())
        news_date = get_news_date(soup)
        news_title = get_news_title(soup)
        news_body = get_news_body(soup)
        return(news_date,news_title,news_body)
    else :
        print(r.status_code)

In [ ]:
with open('./google_search_result_list_TSMC.csv', 'r') as f:
    lines = f.readlines()
print(len(lines))

In [ ]:
c = 0
for line in lines:
    c = c + 1
    print(c)
    time.sleep(1)
    
    url = line.strip()
    data = fetch_page(url)
    if data != None:
        (news_date,news_title,news_body) = data
        filename = news_date.strip()
        filename = filename.replace('.','_')
        filename = filename.replace('/','_')
        filepath = './ctee_TSMC/tmc_' + filename + '.csv'
        with open(filepath,'w') as f:
            f.write(url + '\n')
            f.write(news_date + '\n')
            f.write(news_title + '\n')
            f.write(news_body + '\n')
    else:
        print('return is None')
print('Done')